Basic LSTM

In [70]:
import numpy as np
import pandas as pd
import string
import tensorflow as tf
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
import json
import sklearn
from sklearn import preprocessing as skpp

In [2]:
data = pd.read_csv('./dataset/cleaned_lyrics.csv')

In [3]:
print(data)

# cols: index, song, year, artist, genre, lyrics
# N = 227449 songs

        Unnamed: 0   index                                               song  \
0                0       0                                          ego-remix   
1                1       1                                       then-tell-me   
2                2       2                                            honesty   
3                3       3                                    you-are-my-rock   
4                4       4                                      black-culture   
5                5       5                             all-i-could-do-was-cry   
6                6       6                                 once-in-a-lifetime   
7                7       7                                            waiting   
8                8       8                                          slow-love   
9                9       9                              why-don-t-you-love-me   
10              10      10                                      save-the-hero   
11              11      11  

In [25]:
numpy_data = data['lyrics'].values
max_words = 5000

# create a new Tokenizer
tokenizer = text.Tokenizer(num_words=max_words)
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(numpy_data)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index

with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)

In [31]:
indexed_data = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for txt in numpy_data:
    wordIndices = [dictionary[word] for word in text.text_to_word_sequence(txt)]
    indexed_data.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
indexed_data = np.asarray(indexed_data)

In [91]:
#max lyric length is 6208 at song #9467
#top 10 lyric lengths: [5131 4287 6208 3278 3167 3155 3153 2997 2750 2660]
#for top 1000 lengthiest songs, even first 1000 words seems sufficient
#for top 100 lengthiest songs, first 1500 words seems sufficient
#np.max(np.vectorize(len)(indexed_data))
#temp = np.partition(-np.vectorize(len)(indexed_data), 100)
#result_args = temp[:100]

label_encoder = skpp.LabelEncoder()
indexed_labels = np.array(label_encoder.fit_transform(data['genre'].values))
#label_encoder.inverse_transform(np.array([10, 8])) #to get original genre text back

In [101]:
num_test = 30000

#shuffle data before splitting off test set
random_indexes = numpy.random.permutation(len(indexed_labels))
indexed_data = indexed_data[random_indexes]
indexed_labels = indexed_labels[random_indexes]

X_train = indexed_data[:-num_test]
y_train = indexed_labels[:-num_test]
X_test  = indexed_data[-num_test:]
y_test  = indexed_labels[-num_test:]

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [97]:
# fix random seed for reproducibility
numpy.random.seed(7)
X_train_padded = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test_padded = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [106]:
#vocab size = 336097

[336097, 14, 12, 5061, 5547, 169, 3729, 434, 8548, 12, 881, 623, 10153, 3565, 1, 4756, 11, 246, 2284, 7808, 2656, 146, 1, 5292, 7902, 9, 6, 7054, 1484, 2117, 156, 132, 2401, 11, 213, 35791, 11, 15237, 423, 1, 10657, 188, 12, 15348, 20473, 2702, 1, 4720, 32, 12, 47489, 76808, 525, 11, 4796, 22531, 11, 5172, 29403, 2182, 552, 11, 336098, 20, 2166, 1, 4049, 1343, 146, 1708, 11235, 316, 32067, 2730, 40, 4606, 82, 95, 1762, 29218, 1, 8368, 11, 193, 5406, 32527, 8113, 15207, 4883, 23, 25253, 859, 1, 7412, 11, 29, 2814, 1833, 14471, 17, 1, 9006, 38821, 1825, 1, 332, 11, 1289, 6091, 489, 1, 691, 7902, 2117, 1870, 704]


In [108]:
# truncate and pad input sequences
max_review_length = 1000
# create the model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(11, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train_padded, y_train, nb_epoch=1, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test_padded, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 1000, 32)          10755104  
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_9 (Dense)              (None, 11)                1111      
Total params: 10,809,415
Trainable params: 10,809,415
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
   384/197449 [..............................] - ETA: 2:50:16 - loss: 2.3898 - acc: 0.3776

/anaconda/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


KeyboardInterrupt: 